# 03 – Efficient Frontier

Visualise the full Efficient Frontier with:
- 5,000 simulated random portfolios (background cloud)
- The optimised frontier curve
- Global Minimum Variance Portfolio (GMVP)
- Maximum Sharpe Ratio Portfolio (MSR)
- Capital Market Line (CML)

In [ ]:
import sys; sys.path.insert(0, '..')
from src.data_handler  import load_data, simulate_random_portfolios
from src.optimizer     import min_variance, max_sharpe, efficient_frontier
from src.visualization import plot_efficient_frontier, plot_weights

TICKERS   = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'JPM', 'SPY']
RF        = 0.04
MAX_W     = None   # set to e.g. 0.35 to cap per-asset weight

data      = load_data(TICKERS, '2015-01-01', '2024-12-31', train_end='2023-12-31')
mu, cov   = data['mu'], data['cov']

# ── Compute key portfolios ─────────────────────────────────────────────────────
gmvp = min_variance(mu, cov, max_weight=MAX_W)
msr  = max_sharpe(mu, cov, risk_free_rate=RF, max_weight=MAX_W)
ef   = efficient_frontier(mu, cov, n_points=60, max_weight=MAX_W, risk_free_rate=RF)
rand = simulate_random_portfolios(mu, cov, n_portfolios=5000, risk_free_rate=RF)

print(f'Frontier points: {len(ef)}')
ef[['ret','vol','sharpe']].describe()

In [ ]:
# ── Interactive frontier plot ─────────────────────────────────────────────────
fig = plot_efficient_frontier(
    ef, rand, gmvp, msr,
    tickers=TICKERS,
    risk_free_rate=RF,
    title='Markowitz Efficient Frontier – Tech + Market Portfolio',
)
fig.show()

In [ ]:
# ── Weight charts ─────────────────────────────────────────────────────────────
plot_weights(gmvp['weights'], 'Global Minimum Variance – Weights').show()
plot_weights(msr['weights'],  'Maximum Sharpe Ratio – Weights').show()

In [ ]:
# ── Effect of diversification cap ─────────────────────────────────────────────
import plotly.graph_objects as go

fig2 = go.Figure()
for cap, name, color in [(None, 'Unconstrained', 'navy'), (0.4, 'Max 40%', 'darkorange'), (0.25, 'Max 25%', 'crimson')]:
    ef_c = efficient_frontier(mu, cov, n_points=50, max_weight=cap, risk_free_rate=RF)
    fig2.add_trace(go.Scatter(x=ef_c['vol'], y=ef_c['ret'], mode='lines', name=name,
                              line=dict(color=color, width=2.5)))
fig2.update_layout(title='Efficient Frontier: Impact of Weight Cap',
                   xaxis_title='Volatility', yaxis_title='Return',
                   xaxis_tickformat='.1%', yaxis_tickformat='.1%')
fig2.show()

In [ ]:
# ── Sensitivity: risk-free rate ────────────────────────────────────────────────
import pandas as pd
rows = []
for rf in [0.00, 0.02, 0.04, 0.05, 0.06]:
    m = max_sharpe(mu, cov, risk_free_rate=rf)
    if m['weights'] is not None:
        rows.append({'rf': f'{rf:.0%}', 'Return': m['ret'], 'Vol': m['vol'], 'Sharpe': m['sharpe']})
pd.DataFrame(rows).set_index('rf').style.format({'Return':'{:.2%}','Vol':'{:.2%}','Sharpe':'{:.3f}'})